In [72]:
import os
import json
import pprint
import pickle

import torch
import torch.optim as optim
from tensorboard_logger import Logger as TbLogger
from torch.utils.data import DataLoader

from nets.critic_network import CriticNetwork
from train import train_epoch
from nets.reinforce_baselines import CriticBaseline
from nets.attention_model import AttentionModel
from utils import torch_load_cpu, load_problem, get_inner_model, move_to
from utils.plots import plot_tour
from siam_ensemble import validate
import warnings
warnings.filterwarnings("ignore", category=Warning)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [73]:

import torch
from os import listdir 
from os.path import isfile,join 
from random import sample 

# load the run args
%run options
mypath = './outputs/tsp_20/run_name_20220326T164210/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

condidate_models = []
for name in onlyfiles:
    
    if name != 'args.json' and name[0] == 'v' and int(name.split('-')[1].split('.')[0])> 70:
        condidate_models.append(name)

selected_models = sample(condidate_models,10)
selected_models

['validate-83.pt',
 'validate-95.pt',
 'validate-93.pt',
 'validate-74.pt',
 'validate-86.pt',
 'validate-88.pt',
 'validate_swa-90.pt',
 'validate_swa-80.pt',
 'validate-77.pt',
 'validate-90.pt']

In [74]:
# Set the random seed
torch.manual_seed(opts.seed)

# Optionally configure tensorboard
tb_logger = None
if not opts.no_tensorboard:
    tb_logger = TbLogger(os.path.join(opts.log_dir, "{}_{}".format(opts.problem, opts.graph_size), opts.run_name))

if not os.path.exists(opts.save_dir):
    os.makedirs(opts.save_dir)

# Save arguments so exact configuration can always be found
with open(os.path.join(opts.save_dir, "args.json"), 'w') as f:
    json.dump(vars(opts), f, indent=True)

 # Set the device
opts.device = torch.device("cuda" if opts.use_cuda else "cpu")

In [75]:
problem = load_problem(opts.problem)(
                        p_size = opts.graph_size, # tsp size
                        with_assert = not opts.no_assert) # if checking feasibiliy

TSP with 20 nodes.


In [76]:
model_class = {
    'attention': AttentionModel,
}.get(opts.model, None)
assert model_class is not None, "Unknown model: {}".format(model_class)
                                                           
model = model_class(
    problem = problem,
    embedding_dim = opts.embedding_dim,
    hidden_dim = opts.hidden_dim,
    n_heads = 1, # can increase for better performance
    n_layers = opts.n_encode_layers,
    normalization = opts.normalization,
    device = opts.device,
).to(opts.device)

if opts.use_cuda and torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
    
model

AttentionModel(
  (embedder): EmbeddingNet(
    (embedder): Linear(in_features=2, out_features=128, bias=True)
  )
  (encoder): Sequential(
    (0): MultiHeadAttentionLayer(
      (0): SkipConnection(
        (module): MultiHeadAttention()
      )
      (1): Normalization(
        (normalizer): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): SkipConnection(
        (module): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.001, inplace=False)
          (3): Linear(in_features=128, out_features=128, bias=True)
          (4): Dropout(p=0.001, inplace=False)
        )
      )
      (3): Normalization(
        (normalizer): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): MultiHeadAttentionLayer(
      (0): SkipConnection(
        (module): MultiHeadAttention()
      )
      (1): Normalization(
        (no

In [77]:
def load_model(model_path):
        # Load data from load_path
    load_data = {}
    assert opts.load_path is None or opts.resume is None, "Only one of load path and resume can be given"
    load_path = opts.load_path if opts.load_path is not None else opts.resume
    if load_path is not None:
        print('  [*] Loading data from {}'.format(load_path))
        load_data = torch_load_cpu(load_path)

    # Overwrite model parameters by parameters to load
    model_ = get_inner_model(model)
    model_.load_state_dict({**model_.state_dict(), **load_data.get('model', {})})
    return model
models = []
for path in selected_models:
    models.append(load_model(mypath+path))
models

[AttentionModel(
   (embedder): EmbeddingNet(
     (embedder): Linear(in_features=2, out_features=128, bias=True)
   )
   (encoder): Sequential(
     (0): MultiHeadAttentionLayer(
       (0): SkipConnection(
         (module): MultiHeadAttention()
       )
       (1): Normalization(
         (normalizer): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
       (2): SkipConnection(
         (module): Sequential(
           (0): Linear(in_features=128, out_features=128, bias=True)
           (1): ReLU()
           (2): Dropout(p=0.001, inplace=False)
           (3): Linear(in_features=128, out_features=128, bias=True)
           (4): Dropout(p=0.001, inplace=False)
         )
       )
       (3): Normalization(
         (normalizer): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (1): MultiHeadAttentionLayer(
       (0): SkipConnection(
         (module): MultiHeadAttention()
       )
       (

# Load dataset


In [15]:
ema_avg = lambda averaged_model_parameter, model_parameter, num_averaged: (num_averaged * averaged_model_parameter + model_parameter)/ (num_averaged + 1)
ema_avg(4,2,2)

3.3333333333333335

In [13]:
def avg_fn(averaged_model_parameter, model_parameter, num_averaged):
    return (num_averaged * averaged_model_parameter + model_parameter)/ (num_averaged + 1) 
avg_fn(4,2,2)

3.3333333333333335

In [4]:
import pickle
with open('datasets/tsp_20_10000.pkl','rb') as input_file:
    test_set = pickle.load(input_file)
test_set[0]

tensor([[0.4509, 0.8512],
        [0.3281, 0.4170],
        [0.2536, 0.4560],
        [0.6566, 0.6865],
        [0.0264, 0.5931],
        [0.2647, 0.8877],
        [0.9084, 0.8208],
        [0.3501, 0.1872],
        [0.9299, 0.0127],
        [0.1770, 0.8813],
        [0.1611, 0.9836],
        [0.6462, 0.2009],
        [0.5012, 0.6662],
        [0.4446, 0.8552],
        [0.5352, 0.5726],
        [0.9432, 0.0668],
        [0.6561, 0.2709],
        [0.7097, 0.8725],
        [0.6250, 0.1875],
        [0.6853, 0.0979]])

# Validating the model 


In [79]:
validate(problem, models, test_set[:100], tb_logger, opts)


Validating...


rollout:   0%|                    | 0/1000 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (20) must match the size of tensor b (400) at non-singleton dimension 1

In [81]:
x = torch.ones(20,20)
y = torch.zeros(20,20)

x.add(y) /2

tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5